In [19]:
#베이스라인만들어보자
import numpy as np
import pandas as pd
#train=pd.read_csv("/kaggle/input/playground-series-s4e1/train.csv")
#test=pd.read_csv("/kaggle/input/playground-series-s4e1/test.csv")
#submission=pd.read_csv("/kaggle/input/playground-series-s4e1/sample_submission.csv")
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submission=pd.read_csv("sample_submission.csv")
#ROC이용
print(train.shape,test.shape,submission.shape)

#Exited대신 target사용하겠습니다
train.rename(columns={'Exited':'target'}, inplace=True)
test.rename(columns={'Exited':'target'}, inplace=True)

(165034, 14) (110023, 13) (110023, 2)


In [20]:
#데이터 합치기
all_data=pd.concat([train,test],ignore_index=True)
all_data=all_data.drop('target',axis=1)#타깃값제거

In [21]:
#feature engineering
#간단한 전처리(feature engineering)
df=all_data

#ordinal encoding은 label도 순서지만 내가 원하는 순서대로 인코딩하고 싶을땐 map함수이용해서하거나 아니면 알파벳은 ordinal encoder쓰면
#편함 그냥 라벨은 순서가 고려되긴하는데 막 넣을수있기에 조심해서보고해야함.
#sklearn의 모델이나 xgboost는 반드시 레이블이나 원핫 해줘야함 
#lightgbm ,catboost ,tree기반 ,statsmodel은 알아서 해줌(더미변수화든 원핫이든 레이블이든)

#목적변수는 int형으로 가만히 두기
#Xgboost를 R에서 사용할때는 상관 없지만 Python에서 사용할때는 
#범주형 변수들을 (Object나 Category로 되어 있는 변수) 숫자형 변수로 바꿔줘야 한다.
#그래서 0과1로되어있는 이진변수(int)까지는 숫자형으로 바꿀필요없는듯 여튼
#귀찮은거 많이 없애주고 속도도빠르니 lightgbm쓰나벼?



#중복값이면 1아니면 0인 feature를 df에 추가한다.
df['duple'] = df['CustomerId'].duplicated(keep=False).astype(int)
df['duple']=df['duple'].astype("category")




#인코딩(모델링부분에서 할 것)

#1.HasCrCard,IsActiveMember 2가지는 범주형으로(이미 숫자) 
df['HasCrCard'] = df['HasCrCard'].astype('category')
df['IsActiveMember'] = df['IsActiveMember'].astype('category')

#2.NumOfProducts는 카테고리형으로(이미 숫자고 순서적용)->이미 숫자아니면 뒤에다가 .cat.codes진행
df['NumOfProducts'] = df['NumOfProducts'].astype('category')


#3.Geography와 Gender는 object지만 나중을 위해 숫자로 변환
#3개니까 Geography 원핫 인코딩 진행(순서를 고려해야할경우는 label인코딩을 해야함)
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')
df_encoded = pd.DataFrame(encoder.fit_transform(df[['Geography']]), columns=encoder.get_feature_names_out(['Geography']))
# 기존 데이터프레임에서 'Geography' 열 제거
df = df.drop(['Geography'], axis=1)
# 인코딩된 결과를 기존 데이터프레임에 합치기
df = pd.concat([df, df_encoded], axis=1)


#2개니까 gender는 원핫인코딩필요 없음 레이블 인코디진행 0과1이잖아
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
#레이블을 category로 변환
df['Gender'] = df['Gender'].astype('category')


#4.Surname과 관련된 타깃 인코딩
#from category_encoders import TargetEncoder
# 특성과 타겟을 나누기
#X = df.drop('Exited', axis=1)
#y = df['Exited']
## TargetEncoder 생성 및 인코딩
#encoder = TargetEncoder(cols=['Surname'])
#df= encoder.fit_transform(X, y)
#바꾼 Surname의 feature가 연속이다. object형태로 되어있기에 변환해준다.
#df['Surname_targetencoding'] = df['Surname'].astype('float')


#5.Surname과 관련된 onehot인코딩(속도때매 타깃을쓰고싶지만만..그냥하자)
#plus여기는 lightgbm쓸거니까 그냥 라벨인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Surname'] = label_encoder.fit_transform(df['Surname'])
df['Surname']=df['Surname'].astype("category")

#6.CustomerId 와 관련된 onehot인코딩해야하지만 lightgbm은 그냥 이걸로도 충분
label_encoder = LabelEncoder()
df['CustomerId'] = label_encoder.fit_transform(df['CustomerId'])
df['CustomerId']=df['CustomerId'].astype("category")

#7.id없애기
df = df.drop(['id'], axis=1)

#8.Balance 0인지 아닌지에 대한 feature생성
df['Balance2'] = (df['Balance'] == 0).astype(int)
df['Balance2']=df['Balance2'].astype("category")

all_data=df

In [22]:
#all_data
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275057 entries, 0 to 275056
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   CustomerId         275057 non-null  category
 1   Surname            275057 non-null  category
 2   CreditScore        275057 non-null  int64   
 3   Gender             275057 non-null  category
 4   Age                275057 non-null  float64 
 5   Tenure             275057 non-null  int64   
 6   Balance            275057 non-null  float64 
 7   NumOfProducts      275057 non-null  category
 8   HasCrCard          275057 non-null  category
 9   IsActiveMember     275057 non-null  category
 10  EstimatedSalary    275057 non-null  float64 
 11  duple              275057 non-null  category
 12  Geography_Germany  275057 non-null  float64 
 13  Geography_Spain    275057 non-null  float64 
 14  Balance2           275057 non-null  category
dtypes: category(8), float64(5), int64(

In [23]:
#나중에 필요한 all_feature
all_features=all_data.columns
all_features

Index(['CustomerId', 'Surname', 'CreditScore', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'duple', 'Geography_Germany', 'Geography_Spain',
       'Balance2'],
      dtype='object')

In [24]:
num_train=len(train)#훈련데이터 개수
X=all_data[:num_train]
X_test=all_data[num_train:]

y=train['target']

In [25]:
#8장9장을 잘 해야겠네
#평가지표 계산함수 작성
from sklearn.metrics import roc_auc_score


# ROC AUC 계산
#roc_auc = roc_auc_score(y_true, y_scores)
#y_true는 실제 값 y_scores는 예측 확률값이다.


#Lightgbm용 roc_auc 계산 함수
def roc_auc(preds,dtrain):
    labels=dtrain.get_label()
    return 'roc_auc', roc_auc_score(labels,preds), True

#살펴보면 dtrain이라는 데이터셋의 타깃값을 반환해 labels에 넣고
#preds는 우리가 앞으로 구할 예측 확률값이다.

In [26]:
#oof방식(내가 잘 아는 k번한다음에 마지막에 평균내기)
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)#5개로나눈다

In [27]:
#파라미터
params = {'objective': 'binary',
          'learning_rate': 0.01,
          'force_row_wise': True, #경고문구 없애라
          'random_state': 101}

In [28]:
# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 

In [29]:
import lightgbm as lgb

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정 
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]  # 훈련용 데이터 (iloc 사용)
    X_valid, y_valid = X.iloc[valid_idx], y.iloc[valid_idx]  # 검증용 데이터 (iloc 사용)

    # LightGBM 전용 데이터셋 생성 
    dtrain = lgb.Dataset(X_train, y_train)  # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid)  # LightGBM 전용 검증 데이터셋
    
    # LightGBM 모델 훈련 
    lgb_model = lgb.train(params=params,        # 훈련용 하이퍼파라미터
                          train_set=dtrain,     # 훈련 데이터셋
                          num_boost_round=1000, # 부스팅 반복 횟수
                          valid_sets=dvalid,    # 성능 평가용 검증 데이터셋
                          feval=roc_auc,        # 검증용 평가지표
                          early_stopping_rounds=100, # 조기종료 조건
                          verbose_eval=100)     # 100번째마다 점수 출력
    
    # 테스트 데이터를 활용해 OOF 예측(이게 진짜 결과임 5로나눠서 더해주잖아)
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측(각 폴드마다)->5로 안나누어주는 이유는 평균값을 낼이유가 없잖아 폴드마다 결과를 보여
    #주기만 하면되는데
    oof_val_preds[valid_idx] = lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 auc넓이
    score = roc_auc_score(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} auc넓이 : {score}\n')

######################################## 폴드 1 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] Total Bins 12978
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
Training until validation scores don't improve for 100 rounds


C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorica

[100]	valid_0's binary_logloss: 0.364828	valid_0's roc_auc: 0.883461
[200]	valid_0's binary_logloss: 0.332799	valid_0's roc_auc: 0.888017
[300]	valid_0's binary_logloss: 0.323351	valid_0's roc_auc: 0.889892
[400]	valid_0's binary_logloss: 0.320627	valid_0's roc_auc: 0.890268
Early stopping, best iteration is:
[375]	valid_0's binary_logloss: 0.320928	valid_0's roc_auc: 0.890364
폴드 1 auc넓이 : 0.8903641429145484

######################################## 폴드 2 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] Total Bins 13014
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 15


C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorica

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.361516	valid_0's roc_auc: 0.887264
[200]	valid_0's binary_logloss: 0.328664	valid_0's roc_auc: 0.891322
[300]	valid_0's binary_logloss: 0.318519	valid_0's roc_auc: 0.89331
[400]	valid_0's binary_logloss: 0.315454	valid_0's roc_auc: 0.893777
[500]	valid_0's binary_logloss: 0.314698	valid_0's roc_auc: 0.893635
Early stopping, best iteration is:
[436]	valid_0's binary_logloss: 0.314956	valid_0's roc_auc: 0.893815
폴드 2 auc넓이 : 0.8938147895339214

######################################## 폴드 3 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] Total Bins 12994
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 15


...

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.364393	valid_0's roc_auc: 0.884821
[200]	valid_0's binary_logloss: 0.332045	valid_0's roc_auc: 0.889064
[300]	valid_0's binary_logloss: 0.322625	valid_0's roc_auc: 0.890608
[400]	valid_0's binary_logloss: 0.31969	valid_0's roc_auc: 0.89097
Early stopping, best iteration is:
[383]	valid_0's binary_logloss: 0.319924	valid_0's roc_auc: 0.891022
폴드 3 auc넓이 : 0.891022242131476

######################################## 폴드 4 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 27936, number of negative: 104091
[LightGBM] [Info] Total Bins 12968
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 15


C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorica

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211593 -> initscore=-1.315349
[LightGBM] [Info] Start training from score -1.315349
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.366408	valid_0's roc_auc: 0.878377
[200]	valid_0's binary_logloss: 0.335672	valid_0's roc_auc: 0.882992
[300]	valid_0's binary_logloss: 0.326982	valid_0's roc_auc: 0.884737
[400]	valid_0's binary_logloss: 0.324557	valid_0's roc_auc: 0.885149
[500]	valid_0's binary_logloss: 0.324304	valid_0's roc_auc: 0.884847
Early stopping, best iteration is:
[425]	valid_0's binary_logloss: 0.324295	valid_0's roc_auc: 0.885195
폴드 4 auc넓이 : 0.8851951437820329

######################################## 폴드 5 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 27937, number of negative: 104091
[LightGBM] [Info] Total Bins 13013
[LightGBM] [Info] Number of data points in the train set: 132028, number of used features: 15


C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorica

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211599 -> initscore=-1.315314
[LightGBM] [Info] Start training from score -1.315314
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.362214	valid_0's roc_auc: 0.885908
[200]	valid_0's binary_logloss: 0.329511	valid_0's roc_auc: 0.890058
[300]	valid_0's binary_logloss: 0.320024	valid_0's roc_auc: 0.891769
[400]	valid_0's binary_logloss: 0.317463	valid_0's roc_auc: 0.891839
Early stopping, best iteration is:
[357]	valid_0's binary_logloss: 0.318104	valid_0's roc_auc: 0.891974
폴드 5 auc넓이 : 0.8919740256570283



In [30]:
print('test데이터에대한 roc넓이', roc_auc_score(y, oof_val_preds))

test데이터에대한 roc넓이 0.8903917122394742


In [31]:
submission['Exited']=oof_test_preds
submission.to_csv('submission.csv',index=False)

In [32]:
submission

,id,Exited
0,165034,0.021162
1,165035,0.710880
2,165036,0.030456
3,165037,0.252227
4,165038,0.349166
...,...,...
110018,275052,0.044442
110019,275053,0.135457
110020,275054,0.052541
110021,275055,0.138181
